### University of Stirling
#### Computing Science and Mathematics
#### CSCU9YE - Artificial Intelligence

# Coursework: The Multi-dimensional Knapsack Problem

## Aquiring the problem data

-  This function reads the data of a given problem instance
-  It returns four variables in the the following order, using the variable names as in the mathematical formulation described in the coursework
    - *n*: number of items, 
    - *m*: number of constraints
    - *p*: one dimensional numpy array with the profit coefficients fore ach item 
    - *r*: two dimensional numpy array with the resource coefficients for each item on each constraint/resource
    - *b*: one dimensional numpy array with the constraints right-hand size, that is the bounds or capacities 


In [70]:
import numpy as np

def read_multi_knapsack(fname):
    """
    Reads the data of a multi-knapsack instance

    :param fname: file name with instance data
    :return: n, m, p, r and b (as described above)
    """ 
    profits = []
    with open(fname, 'r') as kfile:
        lines = kfile.readlines()
        
    # convert m,n  to integer varaibles   
    n, m  = [int(d) for d in lines[0].split()]   # convert string data to integers
        
    input_line_cnt = 1    # input lines index after first line
    p = np.empty(0, dtype=np.int64)
    while p.size < n:
        d = np.loadtxt(lines[input_line_cnt].split(),
                       delimiter=" ", dtype=np.int64)
        p = np.append(p, d)
        input_line_cnt += 1

    r = np.empty((0, n), dtype=np.int64)
    for i in range(m):
        lin = np.empty(0, dtype=np.int64)
        while lin.size < n:
            d = np.loadtxt(lines[input_line_cnt].split(),
                           delimiter=" ", dtype=np.int64)
            lin = np.append(lin, d)
            input_line_cnt += 1
        r = np.vstack((r, lin))

    # get the capacities (max of one line space separated integer values)
    b = np.loadtxt(lines[input_line_cnt].split(),
                     delimiter=" ", dtype=np.int64)

    return  n, m, p, r, b, 


In [71]:
## Reading data from a file

data_file_name = "multi_knap_n28_m2.txt"        

n, m, profits, res, cap = read_multi_knapsack(data_file_name)

print("Instance Data:")
print(f"n: {n}.  m: {m}") # amount of items, amount of knapsacks
print(f"Profits: {profits}") # index is item, value is profit, space taken found under same index within resource
print(f"Resources: {res}") # amount of space taken
print(f"Capacities: {cap}") # limit per knapsack, index identifies knapsack


Instance Data:
n: 28.  m: 2
Profits: [ 1898   440 22507   270 14148  3100  4650 30800   615  4975  1160  4225
   510 11880   479   440   490   330   110   560 24355  2885 11748  4550
   750  3720  1950 10500]
Resources: [[ 45   0  85 150  65  95  30   0 170   0  40  25  20   0   0  25   0   0
   25   0 165   0  85   0   0   0   0 100]
 [ 30  20 125   5  80  25  35  73  12  15  15  40   5  10  10  12  10   9
    0  20  60  40  50  36  49  40  19 150]]
Capacities: [500 500]


In [72]:
def evaluate(soln, res, capa): # res values and cap values from 2d arrays. soln = choosen solution
    totalProfits = 0
    totalRes = 0
    AddedProfits = np.array([0] * n)
    AddedRes = np.array([0] * n)

    for i in range(0 , m - 1):
        for j in soln:
            if soln[j] == 1:
                totalRes += res[i][j]
                totalProfits += profits[j]
        if totalRes > capa[i]:
            AddedProfits[i] = 0
        else:
            AddedProfits[i] = totalProfits
        totalProfits = 0
        AddedRes[i] = totalRes
        totalRes = 0
    
    return totalProfits, totalRes # profits = 0 if totalRes > 500


#for i in range(m): # this is just a check to make sure evaluate works, looks fine
  #  print(evaluate([1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
   #                res[i], cap[i]))

In [73]:
def validate(totalProfits, totalRes, capa):
    valid = False
    if totalProfits > 0 and totalRes <= capa:
        valid = True
    return valid
        
print(validate(500, 502, cap[0]))
print(validate(60000000, 400, cap[0]))

False
True


In [74]:
import random as rand

def randomSoln(size):
    soln = []
    
    for i in range(size):
        randResult = rand.randint(0,10)
        if (randResult > 4):
            soln.append(1)
        else:
            soln.append(0)
            
    return soln

print(randomSoln(n))

[0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0]


In [75]:
# so far we have: randomSoln - soln generator
#                 evaluate - gets the results of a soln
#                 validate - checks if the soln is valid

In [76]:
import random

data_file_name = "multi_knap_n28_m2.txt"

n, m, profits, res, cap = read_multi_knapsack(data_file_name)

def neighbour(sol):
    selection = random.randint(0 , n - 1)
    if sol[selection] == 0:
        sol[selection] = 1
    else:
        sol[selection] = 0

    return sol

def hillClimbing(tries, size, neighbourChecks, newSolTries): # int int int int 
    solution = randomSoln(size)
    solSaves = []
    checksSave = neighbourChecks
    solProfits = 0
    solRes = 0
    bestProfits, bestRes = evaluate(solution, res ,cap)

    if validate(bestProfits, bestProfits, cap[0]):
        bestSol = solution
    
    print(f'Original values: \n{solution} val: {bestProfits} weight: {bestRes} \n')
    
    while tries > 0:
        solSaves.append(solution)
        neighbourChecks = checksSave
        while neighbourChecks > 0:
            solution = neighbour(solution)
            solProfits, solRes = evaluate(solution , res , cap)

            if solProfits > bestProfits:
                print(solProfits)
                bestProfits = solProfits
                bestSol = solution
                bestRes = solRes
            elif solProfits == bestProfits and solRes < bestRes:
                bestProfits = solProfits
                bestSol = solution
                bestRes = solRes
                
            #print(f'{solution}  {solProfitsVal}  {solution}  {solRes}') # bug check
            neighbourChecks -= 1
            
        solution = randomSoln(size)
        while solution in solSaves and newSolTries != 0:
            solution = randomSoln(size)
            newSolTries -= 1
            
        if newSolTries == 0:
            break
        # print("loop") # bug check
        tries -= 1
    print(f'New soln tries remaining:{newSolTries} ')
    #print(solSaves) # bug check
    return bestProfits, bestSol, bestRes

bestProfits, bestSol, bestRes = hillClimbing(1000, n, 30, 300) # (tries, size, neighbourChecks, attempts to make new soln)

print(f'The best soln found was {bestSol}. It has a value of {bestProfits} and a weight of {bestRes}')

Original values: 
[0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1] val: 7480 weight: 0 

7920
22776
24674
26572
28470
30368
32266
34164
36062
37960
38564
40022
41480
42938
43654
New soln tries remaining:300 
The best soln found was [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0]. It has a value of 43654 and a weight of 0
